In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path

from insync.db import ListDB
from insync.list import CompletionCommand, ListItem, ListItemContext, ListItemContextType, ListRegistry


In [3]:
descs = ['eggs', 'milk', 'bread', 'butter']

reg = ListRegistry()

for desc in descs:
    i = ListItem(desc, context=ListItemContext('grocery', ListItemContextType.CHECKLIST))
    reg.add(i)

milk_item = list(reg.items)[1]
cmd = CompletionCommand(milk_item.uuid, True)

print("\nList:\n",reg, sep='')
print("Cmd before doing:\n\t", cmd)
reg.do(cmd)
print("Cmd after doing:\n\t", cmd)
print("\nList:\n",reg, sep='')

reg.undo()
print("\nList:\n",reg, sep='')


List:
2024-03-01 eggs @^grocery
2024-03-01 milk @^grocery
2024-03-01 bread @^grocery
2024-03-01 butter @^grocery

Cmd before doing:
	 CompletionCommand(uuid=UUID('018dfd1a-eeb3-77bc-ad45-bebdc49d7585'), completed_new=True, completed_orig=None)
Cmd after doing:
	 CompletionCommand(uuid=UUID('018dfd1a-eeb3-77bc-ad45-bebdc49d7585'), completed_new=True, completed_orig=False)

List:
2024-03-01 eggs @^grocery
x 2024-03-01 milk @^grocery
2024-03-01 bread @^grocery
2024-03-01 butter @^grocery


List:
2024-03-01 eggs @^grocery
2024-03-01 milk @^grocery
2024-03-01 bread @^grocery
2024-03-01 butter @^grocery



# Save and restore from the db

In [ ]:
DB_FILE = Path("scratch.db")
DB_FILE.unlink(missing_ok=True)
db = ListDB(DB_FILE)
db.ensure_tables_created()
db.patch(reg)

In [ ]:
reg2 = db.load()

reg.add(ListItem("cheese", context=ListItemContext('grocery', ListItemContextType.CHECKLIST)))
print("\nOriginal:\n",reg, sep='')
print("\nRestored:\n",reg2, sep='')